What is the Open Closed Principle all about?

<b>OCP = Open for extension, Closed for modification</b>


Lets understand with the help of a scenario: 

Imagine you have some sort of website or application where a user can find certain products that they want to buy. 

A product can be defined in terms of the product name, color and size of the product. 

So lets define some <b>Enums</b> to delineate the different products. 



In [1]:
from enum import Enum

class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3

class Size(Enum):
    SMALL = 1
    MEDIUM = 2
    LARGE = 3

class Product:
    def __init__(self, name, color, size):
        self.size = size
        self.color = color
        self.name = name

#requirement to filter product based on attributes 
class ProductFilter: 
    def filter_by_color(self, products, color):
        for product in products:
            if product.color == color:
                yield product
            
    def filter_by_size(self, products, size):
        for product in products:
            if product.size == size: 
                yield product

What we have done here in this class is truly a violation of the Open Close Principle

<b>Open Closed Principle suggest that when you add a new functionality add it via extenstion not via modification</b>

<b>OCP = Open for extension, Closed for modification</b>


In the above, we modified the exisiting class which before used to have the filter by color then we modified it to have filter by size, which violates the OCP, as before when the class was made we already tested the class with only one filter for size, now instead of modifying that tested class we should extened this class for the new filter and then tested the new class

Lets say now we need to add one more filter for both size and color, with this approach we will again modify the class 

In [2]:
from enum import Enum

class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3

class Size(Enum):
    SMALL = 1
    MEDIUM = 2
    LARGE = 3

class Product:
    def __init__(self, name, color, size):
        self.size = size
        self.color = color
        self.name = name

#requirement to filter product based on attributes 
class ProductFilter: 
    def filter_by_color(self, products, color):
        for product in products:
            if product.color == color:
                yield product
            
    def filter_by_size(self, products, size):
        for product in products:
            if product.size == size: 
                yield product

    def filter_by_color_and_size(self, products, size, color):
        for product in products:
            if product.color == color and product.size == size:
                yield product



We again modified the exisiting class, this might break the system. 

This can work, but is definitley not scalalable 

Along with the violation of the OCP, we are also causing something known as <b>State Space Explosion</b>


So the 2 criteria filters are giving us a total of 3 possible methods and possibly more.. 
3 criteria filter will give us 7 different methods easily 

So it is definitely not scalable. 

So now we are going to rewrite this in such a way that it does not break the OCP and will also look at an enterprise pattern(<b>Specification</b>) and use it 

In [19]:
#Specification - Design Pattern 
#the whole idea of this pattern is that we extend 
from enum import Enum

class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3

class Size(Enum):
    SMALL = 1
    MEDIUM = 2
    LARGE = 3

class Product:
    def __init__(self, name, color, size):
        self.size = size
        self.color = color
        self.name = name

# 1. Implement the base classes 
#Specification is a class which determines whether or not a particular item satisifies a particular criterion
class Specification:
    def is_satisfied(self, item):
        pass    #this is just a base class, you are meant to over ride this 

    def __and__(self, other):
        return AndSpecification(self, other)

class Filter:
    def filter(self, items, spec):
        pass

#2. Now implement the features 
#lets implement filtering by colors and size
class ColorSpecification(Specification):
    def __init__(self, color):
        self.color = color
    
    def is_satisfied(self, item):
        return item.color == self.color
    
class SizeSpecification(Specification):
    def __init__(self, size):
        self.size = size

    def is_satisfied(self, item):
        return item.size == self.size
    
class AndSpecification(Specification):
    def __init__(self, *args):
        self.args = args
    
    def is_satisfied(self, item):
        return all(map(lambda spec: spec.is_satisfied(item), self.args))
    
class BetterFilter(Filter):
    def filter(self, items, spec):
        for item in items:
            if spec.is_satisfied(item):
                yield item 


In [22]:
#testing both 

if __name__ == "__main__":

    apple = Product("APPLE", Color.RED, Size.SMALL)
    tree = Product("TREE", Color.GREEN, Size.LARGE)
    room = Product("ROOM", Color.BLUE, Size.LARGE)

    products = [apple, tree, room]

    #old approach 
    pf = ProductFilter()
    print("Green Products (Old): ")
    for p in pf.filter_by_color(products, Color.GREEN):
        print(f'{p.name} is green')

    
    bf = BetterFilter()
    print("Green Products(New): ")

    green = ColorSpecification(Color.GREEN)
    for p in bf.filter(products, green):
        print(f'{p.name} is green')

    large = SizeSpecification(Size.LARGE)
    for p in bf.filter(products, large):
        print(f'{p.name} is large')

    large_blue = large & ColorSpecification(Color.BLUE)
    for p in bf.filter(products, large_blue):
        print(f'{p.name} is large and blue')




Green Products (Old): 
TREE is green
Green Products(New): 
TREE is green
TREE is large
ROOM is large
ROOM is large and blue
